In [1]:
from qdrant_client import QdrantClient, models
import os
from dotenv import load_dotenv

load_dotenv()   

True

## Initialize Client

In [2]:
client = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))

# For Colab:
# from google.colab import userdata
# client = QdrantClient(url=userdata.get("QDRANT_URL"), api_key=userdata.get("QDRANT_API_KEY"))

## Create Collection

In [3]:
collection_name = "day0_first_system"
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=4, distance=models.Distance.COSINE),
)

# Create payload index right after creating the collection and before uploading any data to enable filtering.
# If you add it later, HNSW won't rebuild automatically—bump ef_construct (e.g., 100→101) to trigger a safe rebuild.
client.create_payload_index(
    collection_name=collection_name,
    field_name="category",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

## Insert Points

In [4]:
points=[
    models.PointStruct(
        id=1,
        vector=[0.9, 0.1, 0.1, 0.8], # High affordability, high innovation
        payload={"name": "Budget Smartphone", "category": "electronics", "price": 299},
    ),
    models.PointStruct(
        id=2,
        vector=[0.2, 0.9, 0.8, 0.5], # High quality, high popularity
        payload={"name": "Bestselling Novel", "category": "books", "price": 19},
    ),
    models.PointStruct(
        id=3,
        vector=[0.8, 0.3, 0.2, 0.9], # High affordability, high innovation (similar to ID 1)
        payload={"name": "Smart Home Hub", "category": "electronics", "price": 89},
    ),
    # Add 2-5 more points to experiment with...
]

client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

## Test Searches

In [5]:
# Define a query vector for "affordable and innovative"
query_vector = [0.85, 0.2, 0.1, 0.9]

# 1. Basic similarity search
basic_results = client.query_points(collection_name, query=query_vector)

# 2. Filtered search (only find electronics)
filtered_results = client.query_points(
    collection_name,
    query=query_vector,
    query_filter=models.Filter(
        must=[models.FieldCondition(key="category", match=models.MatchValue(value="electronics"))]
    ),
)
print("Filtered search results:", filtered_results)

Filtered search results: points=[ScoredPoint(id=1, version=3, score=0.9933038, payload={'name': 'Budget Smartphone', 'category': 'electronics', 'price': 299}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=3, score=0.9928858, payload={'name': 'Smart Home Hub', 'category': 'electronics', 'price': 89}, vector=None, shard_key=None, order_value=None)]
